## --- Day 5: If You Give A Seed A Fertilizer ---
You take the boat and find the gardener right where you were told he would be: managing a giant "garden" that looks more to you like a farm.

"A water source? Island Island is the water source!" You point out that Snow Island isn't receiving any water.

"Oh, we had to stop the water because we ran out of sand to filter it with! Can't make snow with dirty water. Don't worry, I'm sure we'll get more sand soon; we only turned off the water a few days... weeks... oh no." His face sinks into a look of horrified realization.

"I've been so busy making sure everyone here has food that I completely forgot to check why we stopped getting more sand! There's a ferry leaving soon that is headed over in that direction - it's much faster than your boat. Could you please go check it out?"

You barely have time to agree to this request when he brings up another. "While you wait for the ferry, maybe you can help us with our food production problem. The latest Island Island Almanac just arrived and we're having trouble making sense of it."

The almanac (your puzzle input) lists all of the seeds that need to be planted. It also lists what type of soil to use with each kind of seed, what type of fertilizer to use with each kind of soil, what type of water to use with each kind of fertilizer, and so on. Every type of seed, soil, fertilizer and so on is identified with a number, but numbers are reused by each category - that is, soil 123 and fertilizer 123 aren't necessarily related to each other.

For example:

    seeds: 79 14 55 13

    seed-to-soil map:
    50 98 2
    52 50 48

    soil-to-fertilizer map:
    0 15 37
    37 52 2
    39 0 15

    fertilizer-to-water map:
    49 53 8
    0 11 42
    42 0 7
    57 7 4

    water-to-light map:
    88 18 7
    18 25 70

    light-to-temperature map:
    45 77 23
    81 45 19
    68 64 13

    temperature-to-humidity map:
    0 69 1
    1 0 69

    humidity-to-location map:
    60 56 37
    56 93 4
The almanac starts by listing which seeds need to be planted: seeds 79, 14, 55, and 13.

The rest of the almanac contains a list of maps which describe how to convert numbers from a source category into numbers in a destination category. That is, the section that starts with seed-to-soil map: describes how to convert a seed number (the source) to a soil number (the destination). This lets the gardener and his team know which soil to use with which seeds, which water to use with which fertilizer, and so on.

Rather than list every source number and its corresponding destination number one by one, the maps describe entire ranges of numbers that can be converted. Each line within a map contains three numbers: the destination range start, the source range start, and the range length.

Consider again the example seed-to-soil map:

    50 98 2
    52 50 48
The first line has a destination range start of 50, a source range start of 98, and a range length of 2. This line means that the source range starts at 98 and contains two values: 98 and 99. The destination range is the same length, but it starts at 50, so its two values are 50 and 51. With this information, you know that seed number 98 corresponds to soil number 50 and that seed number 99 corresponds to soil number 51.

The second line means that the source range starts at 50 and contains 48 values: 50, 51, ..., 96, 97. This corresponds to a destination range starting at 52 and also containing 48 values: 52, 53, ..., 98, 99. So, seed number 53 corresponds to soil number 55.

Any source numbers that aren't mapped correspond to the same destination number. So, seed number 10 corresponds to soil number 10.

So, the entire list of seed numbers and their corresponding soil numbers looks like this:

    seed  soil
    0     0
    1     1
    ...   ...
    48    48
    49    49
    50    52
    51    53
    ...   ...
    96    98
    97    99
    98    50
    99    51
With this map, you can look up the soil number required for each initial seed number:

    Seed number 79 corresponds to soil number 81.
    Seed number 14 corresponds to soil number 14.
    Seed number 55 corresponds to soil number 57.
    Seed number 13 corresponds to soil number 13.
The gardener and his team want to get started as soon as possible, so they'd like to know the closest location that needs a seed. Using these maps, find the lowest location number that corresponds to any of the initial seeds. To do this, you'll need to convert each seed number through other categories until you can find its corresponding location number. In this example, the corresponding types are:

    Seed 79, soil 81, fertilizer 81, water 81, light 74, temperature 78, humidity 78, location 82.
    Seed 14, soil 14, fertilizer 53, water 49, light 42, temperature 42, humidity 43, location 43.
    Seed 55, soil 57, fertilizer 57, water 53, light 46, temperature 82, humidity 82, location 86.
    Seed 13, soil 13, fertilizer 52, water 41, light 34, temperature 34, humidity 35, location 35.
So, the lowest location number in this example is 35.

What is the lowest location number that corresponds to any of the initial seed numbers?

In [9]:
import pandas as pd
import numpy as np

In [10]:
data = ['seeds: 79 14 55 13',
'',
'seed-to-soil map:',
'50 98 2',
'52 50 48',
'',
'soil-to-fertilizer map:',
'0 15 37',
'37 52 2',
'39 0 15',
'',
'fertilizer-to-water map:',
'49 53 8',
'0 11 42',
'42 0 7',
'57 7 4',
'',
'water-to-light map:',
'88 18 7',
'18 25 70',
'',
'light-to-temperature map:',
'45 77 23',
'81 45 19',
'68 64 13',
'',
'temperature-to-humidity map:',
'0 69 1',
'1 0 69',
'',
'humidity-to-location map:',
'60 56 37',
'56 93 4']

In [11]:
# with open('day_05_input.txt') as f:
#     data = f.readlines()
# data = [dat.strip('\n') for dat in data]
# data[:5]

In [12]:
seeds = data[0].split(': ')[1:]
seeds = seeds[0].split(' ')
seeds = [int(seed) for seed in seeds]
seeds

[79, 14, 55, 13]

In [13]:
map_names = ['seed-to-soil','soil-to-fertilizer','fertilizer-to-water','water-to-light',
            'light-to-temperature','temperature-to-humidity','humidity-to-location']

In [14]:
maps = {}

for name in map_names:
    
    save = False
    for line in data:
        if line.startswith(name):
            save = True
            continue
        if line == '':
            save = False
        if save:
            if name not in maps:
                maps[name] = []
            maps[name].append(line)

In [15]:
one_map = maps['seed-to-soil']
one_map

['50 98 2', '52 50 48']

In [16]:
all_builds = []
max_of_all = 0 

for mapping in maps:
    one_map = maps[mapping]
    
    build_it = []

    for one_line in one_map:
        one_line = one_line.split()
        one_line = [int(numb) for numb in one_line]
        print(one_line)
        source = one_line[1]
        destination = one_line[0]
        ranged = one_line[2]

        temp_source = np.arange(source,source+ranged)
        temp_destination = np.arange(destination,destination+ranged)
        
        attempt = np.vstack([temp_source, temp_destination]).T.tolist()
        build_it.extend(attempt)
            
    print()
    all_builds.append(build_it)
    
# all_builds

[50, 98, 2]
[52, 50, 48]

[0, 15, 37]
[37, 52, 2]
[39, 0, 15]

[49, 53, 8]
[0, 11, 42]
[42, 0, 7]
[57, 7, 4]

[88, 18, 7]
[18, 25, 70]

[45, 77, 23]
[81, 45, 19]
[68, 64, 13]

[0, 69, 1]
[1, 0, 69]

[60, 56, 37]
[56, 93, 4]



In [ ]:
max_of_all = 0 

for build in all_builds:
    build_max = np.array(build).flatten().max()
    if build_max > max_of_all:
        max_of_all = build_max
        
counter = pd.DataFrame(np.arange(max_of_all+1))
counter.tail()

In [12]:
all_builds_df = pd.DataFrame()

for i, build in enumerate(all_builds):
#     print(i)
    temp_df = pd.DataFrame(build, columns=[i,i+1])
#     print(temp_df.head())
    
    if i == 0:
        all_builds_df = pd.merge(counter, temp_df, how='outer', on=i)
        
    else:
        all_builds_df = pd.merge(all_builds_df, temp_df, how='outer', on=i)

    all_builds_df[i+1] = np.where(all_builds_df[i+1].isnull(), all_builds_df[i], all_builds_df[i+1])
    
all_builds_df

,0,1,2,3,4,5,6,7
0,0,0.0,39.0,28.0,21.0,21.0,22.0,22.0
1,1,1.0,40.0,29.0,22.0,22.0,23.0,23.0
2,2,2.0,41.0,30.0,23.0,23.0,24.0,24.0
3,3,3.0,42.0,31.0,24.0,24.0,25.0,25.0
4,4,4.0,43.0,32.0,25.0,25.0,26.0,26.0
...,...,...,...,...,...,...,...,...
95,95,97.0,97.0,97.0,97.0,65.0,66.0,70.0
96,96,98.0,98.0,98.0,98.0,66.0,67.0,71.0
97,97,99.0,99.0,99.0,99.0,67.0,68.0,72.0
98,98,50.0,35.0,24.0,94.0,62.0,63.0,67.0


In [13]:
all_builds_df

,0,1,2,3,4,5,6,7
0,0,0.0,39.0,28.0,21.0,21.0,22.0,22.0
1,1,1.0,40.0,29.0,22.0,22.0,23.0,23.0
2,2,2.0,41.0,30.0,23.0,23.0,24.0,24.0
3,3,3.0,42.0,31.0,24.0,24.0,25.0,25.0
4,4,4.0,43.0,32.0,25.0,25.0,26.0,26.0
...,...,...,...,...,...,...,...,...
95,95,97.0,97.0,97.0,97.0,65.0,66.0,70.0
96,96,98.0,98.0,98.0,98.0,66.0,67.0,71.0
97,97,99.0,99.0,99.0,99.0,67.0,68.0,72.0
98,98,50.0,35.0,24.0,94.0,62.0,63.0,67.0


In [14]:
for seed in seeds:
    print(seed, all_builds_df.iloc[seed][7])

79 82.0
14 43.0
55 86.0
13 35.0
